# Rasters con valores bioclimáticos y precipitación anual

## bio *Bioclimáticos*
## prec *Precipitación*


In [2]:
# Librerías y paquetes necesarios para poder operar los rasters.
#!pip install geopandas rasterio pandas tqdm

import geopandas as gpd
import pandas as pd
import rasterio
import glob
from tqdm import tqdm

   ---------------------------------------- 0.0/25.4 MB ? eta -:--:--
   ---------------------------------------- 0.1/25.4 MB 2.3 MB/s eta 0:00:11
    --------------------------------------- 0.6/25.4 MB 7.5 MB/s eta 0:00:04
   --- ------------------------------------ 2.4/25.4 MB 19.5 MB/s eta 0:00:02
   ------- -------------------------------- 4.7/25.4 MB 27.4 MB/s eta 0:00:01
   ---------- ----------------------------- 7.0/25.4 MB 31.7 MB/s eta 0:00:01
   -------------- ------------------------- 9.2/25.4 MB 34.5 MB/s eta 0:00:01
   ----------------- ---------------------- 11.4/25.4 MB 50.1 MB/s eta 0:00:01
   --------------------- ------------------ 13.6/25.4 MB 50.4 MB/s eta 0:00:01
   ------------------------ --------------- 15.9/25.4 MB 50.4 MB/s eta 0:00:01
   ---------------------------- ----------- 18.1/25.4 MB 46.7 MB/s eta 0:00:01
   -------------------------------- ------- 20.4/25.4 MB 50.4 MB/s eta 0:00:01
   ----------------------------------- ---- 22.7/25.4 MB 50.4 MB/s et

In [3]:
# Rutas donde tengo almacenada la información climatologica
root_dir = "G:/Capas/worlclim/1x1"

In [18]:
# Listando los archivos .tif en subcarpetas
raster_files = glob.glob(root_dir + "/**/*.tif", recursive=True)
raster_files = [f for f in raster_files if "bio" in f.lower() or "prec" in f.lower()]

# Creando los acrónimos automáticos (nombre del archivo sin extensión) para crear el diccionario
acronyms = [f.split("\\")[-1].replace(".tif", "") for f in raster_files]  # Windows path
# Si usas Linux/Mac cambia a: f.split("/")[-1]


In [19]:
# Cargargando accesiones que ya fueron limpiadas (ejemplo CSV con lon y lat)
accesiones = pd.read_csv("E:/rasters/accesiones.csv")

In [20]:
# Convirtiendo las accesiones accesiones a GeoDataFrame
gdf = gpd.GeoDataFrame(
    accesiones,
    geometry=gpd.points_from_xy(accesiones.DECLONGITUDE, accesiones.DECLATITUDE),
    crs="EPSG:4326"
)

In [21]:
# Extrayendo los valores de los rasters en puntos
def extraer_raster(rfile, gdf):
    with rasterio.open(rfile) as src:
        coords = [(x,y) for x,y in zip(gdf.geometry.x, gdf.geometry.y)]
        vals = [val[0] if val is not None else None for val in src.sample(coords)]
    return vals

In [22]:
# Extrayendo los valores de todos los rasters 
valores = pd.DataFrame()
for i, rfile in enumerate(tqdm(raster_files, desc="Extrayendo rasters")):
    try:
        valores[acronyms[i]] = extraer_raster(rfile, gdf)
    except Exception as e:
        print(f"⚠️ Error en {rfile}: {e}")
        valores[acronyms[i]] = [None] * len(gdf)

Extrayendo rasters: 100%|██████████████████████████████████████████████████████████████████████████████████████████| 31/31 [01:59<00:00,  3.86s/it]


In [23]:
# Uniendo las accesiones con los valores climáticos valores climáticos
resultado = pd.concat([accesiones, valores], axis=1)

In [24]:

# Creando metadata oficial (bio1–bio19 + mensuales)
bio_metadata = pd.DataFrame({
    "Acronym": [f"bio{i}" for i in range(1,20)],
    "Unidad": ["°C"]*11 + ["mm"]*8,
    "Descripcion": [
        "Temperatura media anual",
        "Rango medio diurno (Tmax–Tmin)",
        "Isotermalidad (bio2/bio7 ×100)",
        "Estacionalidad de la temperatura (SD ×100)",
        "Temperatura máxima del mes más cálido",
        "Temperatura mínima del mes más frío",
        "Rango anual de temperatura (bio5–bio6)",
        "Temperatura media del trimestre más húmedo",
        "Temperatura media del trimestre más seco",
        "Temperatura media del trimestre más cálido",
        "Temperatura media del trimestre más frío",
        "Precipitación anual",
        "Precipitación del mes más húmedo",
        "Precipitación del mes más seco",
        "Estacionalidad de la precipitación (CV)",
        "Precipitación del trimestre más húmedo",
        "Precipitación del trimestre más seco",
        "Precipitación del trimestre más cálido",
        "Precipitación del trimestre más frío"
    ]
})

In [26]:
# Generando la tbala de los meses con unidades y descripción para las precipitaciones
def make_monthly(prefix, unidad, desc):
    return pd.DataFrame({
        "Acronym": [f"{prefix}{i}" for i in range(1,13)],
        "Unidad": unidad,
        "Descripcion": [f"{desc} de {m}" for m in [
            "enero","febrero","marzo","abril","mayo","junio",
            "julio","agosto","septiembre","octubre","noviembre","diciembre"
        ]]
    })

prec  = make_monthly("prec", "mm", "Precipitación")
#srad  = make_monthly("srad", "kJ m-2 día-1", "Radiación solar")
#tavg  = make_monthly("tavg", "°C", "Temperatura media")
#tmax  = make_monthly("tmax", "°C", "Temperatura máxima")
#tmin  = make_monthly("tmin", "°C", "Temperatura mínima")
#vapr  = make_monthly("vapr", "kPa", "Presión de vapor")
#wind  = make_monthly("wind", "m/s", "Velocidad del viento")

metadata_oficial = pd.concat([bio_metadata, prec],ignore_index=True)#, srad, tavg, tmax, tmin, vapr, wind], ignore_index=True)

In [27]:
# Creando la metadata final enlazando con archivos cargados
metadata_final = pd.DataFrame({
    "Acronym": acronyms,
    "Variable": [f.split("\\")[-1] for f in raster_files],  # si es Linux usar split("/")
    "Path": raster_files
}).merge(metadata_oficial, on="Acronym", how="left")

In [28]:

# Guardando los resultados de la metadata y de las accesiones con variables climáticas resultados
#resultado.to_excel("E:/rasters/accesiones_con_worldclim.xlsx", index=False)
resultado.to_csv("E:/rasters/accesiones_con_worldclim.csv", index=False, encoding="utf-8")
metadata_final.to_csv("E:/rasters/metadata_worldclim.csv", index=False, encoding="utf-8")

print("✅ Listo: accesiones + variables climáticas guardadas, con metadata completa.")


✅ Listo: accesiones + variables climáticas guardadas, con metadata completa.


In [29]:
resultado.head()

,INSTCODE,DOI,ACCENUMB,HISTORIC,CURATION,GENUS,SPECIES,SPAUTHOR,SUBTAXA,SUBTAUTHOR,...,prec_11,prec_12,prec_2,prec_3,prec_4,prec_5,prec_6,prec_7,prec_8,prec_9
0,COL003,10.18730/JKECH,G16570,False,FULL,Phaseolus,vulgaris,NaN,NaN,NaN,...,31,40,38,49,30,24,7,4,3,3
1,COL003,10.18730/JKEKR,G16576,False,FULL,Phaseolus,vulgaris,NaN,NaN,NaN,...,60,75,69,98,65,34,2,1,1,1
2,COL003,10.18730/JZXX0,G990,False,FULL,Phaseolus,vulgaris,NaN,NaN,NaN,...,37,66,37,41,16,5,0,0,0,0
3,COL003,10.18730/JZXWU,G989,False,FULL,Phaseolus,vulgaris,NaN,NaN,NaN,...,24,48,59,43,5,0,0,0,0,1
4,COL003,10.18730/JZXT$,G987,False,FULL,Phaseolus,vulgaris,NaN,NaN,NaN,...,13,40,34,40,15,6,1,1,1,0
